# Laboratorio # 4: Regresión multivariada

Objetivo: Programar una regresión multivariada


1. Para simular un conjunto de características $x_1$ , $x_2$,..., $x_n$ trabajaremos en la primera parte con dos características de datos aleatorios que presentan un plano y mostraremos que los párametros optimizados se corresponden con el valor esperado.

- Definir la ecuación  $y = 2.1*x_1 - 3.1*x_2$, y generar números aleatorios que pertenecen al plano.

- Realizar un diagrama 3D de los puntos generados aleatoriamente.


Nuestro objetivo será encontrar los valores $\theta_0 = 0, \theta_1=2.1, \theta_1=3.1$ que mejor ajustan el plano, empleando cálculos vectorizados.

2. Inicializar conjunto de parámetros $\Theta$ de manera aleatoria.
3. Construir la matrix X con dimensiones $(n+1, m)$, m es el numero de datos de entrenamiento y (n) el número de caracteristicas.
4. Calcular la función de coste(revise cuidosamente las dimensiones de cada matriz):

  - $h = \Theta^{T} X $
  - $\Lambda= (h -Y) $
  - $\Lambda= [\Lambda_1,\Lambda_2, ...,\Lambda_m]$
  - $J = \frac{1}{2m} \sum_{i}^m \Lambda_i^2 $

5. Aplicar el gradiente descendente:
  - Encontrar el gradiente.
    $\nabla J = \Lambda \cdot X^T$
  
  - Actualizar los nuevos parametros:
    $\Theta_{n+1}=\Theta_{n}-\alpha\nabla J$


6. Iterar para encontrar los valores $\Theta$ que se ajustan el plano.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import plotly.graph_objects as go

Se inicia definiendo el plano y se toman puntos aleatorios sobre este pero con ruido aleatorio para simular condiciones reales.

In [40]:
y = lambda x: 2.1*x[0] - 3.1*x[1]
N = 500

X1 =np.array([random.uniform(-10,10) for i in range(N)])
X2 =np.array([random.uniform(-10,10) for i in range(N)])

X = np.array([X1, X2])
Y = y(X) + 5*np.random.uniform(-1,1, size=len(X1))

xs = np.linspace(-10, 10, 100)
ys = np.linspace(-10, 10, 100)
X1s, X2s = np.meshgrid(xs, ys)
Ys = y(np.array([X1s, X2s]))

fig = go.Figure(go.Surface(x=X1s, y=X2s, z=Ys))
fig = fig.add_trace(go.Scatter3d(x=X[0], y=X[1], z=Y, mode='markers', marker=dict(size=4, color='white')))
fig.show()

Ahora creamos una clase que permite aplicar la regresión lineal sobre el conjunto de datos; a esta se le entrega el arreglo de las caracteristicas y el de los valores esperados.
<!-- -->
Inicialmente se construye la matríz `X` de manera adecuada, teniendo en cuenta que su primer fila son unos para así poder tener en cuenta el parámetro independiente. En la rutina del modelo (`model`) se define la métrica Euclidiana, que se usará para calcular la función de coste en la rutina `cost`, seguido de la rutina `gradD` que aplíca el gradiente descendente; finalmente la rutina `fit` realiza una iteración sobre las anteriores con el fin de encontrar los parámetros que mejor describen el conjunto de datos entregados.
<!-- -->


In [42]:
class ML_Regression():
    def __init__(self, X, Y):
        df = pd.DataFrame(np.column_stack(X))
        df['ones'] = 1
        cols = ['ones'] + [c for c in df.columns if c != 'ones']
        df = df[cols]   # columnas en el orden correcto

        # forma correcta de la matríz de características
        self.X = df.values.T
        self.Y = Y
        n = len(self.X)
        self.m = len(self.Y)
        self.thetas = np.random.random(n).reshape(n,1)
    

    def model(self):
        self.L = self.thetas.T@self.X - self.Y   # métrica Euclidiana

    def cost(self):
        self.J = np.sum(self.L**2)/(2*self.m)
    
    def gradD(self, alpha):
        
        grad = np.dot(self.L,self.X.T).T
        self.thetas = self.thetas - alpha*grad/self.m

    def fit(self, alpha):

        for i in range(0, 1000):
            self.model()
            self.cost()
            self.gradD(alpha)

Procedemos con una prueba del funcionamiento con los datos construidos.

In [43]:
r = ML_Regression(X, Y)
r.fit(0.01)
r.thetas

print(f'''
Parametros ajustados:

      θ₀ = {r.thetas[0][0]:.4f}
      θ₁ = {r.thetas[1][0]:.4f}
      θ₂ = {r.thetas[2][0]:.4f}

'''
)


Parametros ajustados:

      θ₀ = -0.0575
      θ₁ = 2.1411
      θ₂ = -3.1495




### *Juan José Pérez Avendaño*